In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from numpy import random
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp

In [6]:
model = load_model('models/facialPointsPredictor.h5')
model_emotions = load_model('models/emotionPredictor.h5')
emotion_dict = {0: 'Ira', 1: 'Asco', 2: 'Tristeza', 3: 'Felicidad', 4: 'Sorpresa'}

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing_styles = mp.solutions.drawing_styles

In [7]:
class SetFiltersActions:

    def __init__(self):
        self.last_emotion_predicted = ''
        self.second_frame = None
        self.random_filter = 'filtros/sunglasses.png'

    def get_points(self, image):
        image = tf.reshape(image, (96, 96, 1))
        points = model.predict(np.array([image]))[0]

        return points[0::2], points[1::2]


    def enumerate_points(self, points: list[tuple]):
        """Enumerate points in a list of tuples each tuple item is one more index"""
        return list(enumerate(points))

    def draw_rectangles(self, frame):
        cv2.rectangle(frame, (0, 0), (100, 100), (255, 0, 0), -1)
        cv2.putText(frame, 'Change FT', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        cv2.rectangle(frame, (frame.shape[1] - 100, 0), (frame.shape[1], 100), (255, 0, 0), -1)
        cv2.putText(frame, 'Emotion', (frame.shape[1] - 80, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    def set_glasses_filter(self, points, color_face_redim, filter_weight, filter_height, region_no_transparente, filter_resized):
        try:
            color_face_redim[int(points[9][1]):int(points[9][1]) + filter_height,
            int(points[9][0]): int(points[9][0]) + filter_weight, :][region_no_transparente] = filter_resized[:, :, :3][
                region_no_transparente]
        except:
            color_face_redim[int(points[7][1]):int(points[7][1]) + filter_height,
            int(points[7][0]): int(points[7][0]) + filter_weight, :][region_no_transparente] = filter_resized[:, :, :3][
                region_no_transparente]

    def set_moustache_filter(self, points, color_face_redim, filter_weight, filter_height, region_no_transparente, filter_resized):
        color_face_redim[int(points[10][1]):int(points[10][1]) + filter_height,
        int(points[5][0]): int(points[5][0]) + filter_weight, :][region_no_transparente] = filter_resized[:, :, :3][
                region_no_transparente]

    def get_filter(self, filter_image, points, color_face_redim, filter_weight, filter_height, original_shape):
        if filter_weight > 0 and filter_height > 0:
            filter_resized = cv2.resize(filter_image, (filter_weight, filter_height), interpolation=cv2.INTER_CUBIC)

            region_no_transparente = filter_resized[:, :, :3] != 0

            try:
                if 'moustache' in self.random_filter:
                    self.set_moustache_filter(points, color_face_redim, filter_weight, filter_height, region_no_transparente, filter_resized)
                else:
                    self.set_glasses_filter(points, color_face_redim, filter_weight, filter_height, region_no_transparente, filter_resized)
            except:
                pass

            cv2.imshow('filter', color_face_redim)

            color_face = cv2.resize(color_face_redim, original_shape, interpolation=cv2.INTER_CUBIC)

            return color_face

    def get_glasses_filter_dimensions(self, points):
        ancho_gafas = int((points[7][0] - points[9][0]))
        alto_gafas = int((points[10][1] - points[8][1]))

        if ancho_gafas > 0 and alto_gafas > 0:
            return points, (ancho_gafas, alto_gafas)

        ancho_gafas = int((points[9][0] - points[7][0]))
        alto_gafas = int((points[10][1] - points[8][1]))

        if ancho_gafas > 0 and alto_gafas > 0:
            return points, (ancho_gafas, alto_gafas)

        return points, False

    def get_moustache_filter_dimensions(self, points):
        ancho = int((points[7][0] - points[9][0]))
        alto = int((points[14][1] - points[10][1]))

        print(1, ancho, alto)
        if ancho > 0 and alto > 0:
            return points, (ancho, alto)

        return points, False

    def get_filter_dimensions(self, frame_cara):

        points_x, points_y = self.get_points(frame_cara)
        points = list(zip(points_x, points_y))
        print(self.random_filter)

        if 'moustache' in self.random_filter:
            return self.get_moustache_filter_dimensions(points)

        return self.get_glasses_filter_dimensions(points)

    def hand_detection(self, frame):
        results = hands.process(frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())

            return results.multi_hand_landmarks[0].landmark

    def must_change_filter(self, hand_landmarks):
        if hand_landmarks is not None:
            if hand_landmarks[8].y * frame.shape[0] <= 110.0 and hand_landmarks[8].x * frame.shape[1] <= 100.0:
                return True

        return False

    def must_predict_emotion(self, hand_landmarks):
        if hand_landmarks is not None:
            if hand_landmarks[8].y * frame.shape[0] <= 110.0 and hand_landmarks[8].x * frame.shape[1] >= frame.shape[
                1] - 100:
                return True

        return False

    def predict_emotion_and_draw_emotion(self,frame_cara):
        frame_cara = tf.reshape(frame_cara, (-1, 105, 105, 1))

        return emotion_dict[int(np.argmax(model_emotions.predict(frame_cara)))]

    def draw_points(self, points,x,y,w,h):
         for index, point in self.enumerate_points(points):
            #resize points coordinates to original dataframe
            point = (point[0] * w / 96, point[1] * h / 96)

            #draw cirlce
            cv2.circle(self.second_frame, (int(point[0] + x), int(point[1] + y)), 1, (255, 255, 255), 3)

            #draw index
            cv2.putText(self.second_frame, str(index), (int(point[0] + x), int(point[1] + y)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    def detect_faces(self, frame, hand_landmarks):
        faces = faceCascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
        filter_image = cv2.imread(self.random_filter, cv2.IMREAD_UNCHANGED)

        for (x, y, w, h) in faces:
            gray_face = gray[y:y + h, x:x + w]
            color_face = frame[y:y + h, x:x + w]
            frame_cara = cv2.resize(gray_face, (96, 96)) / 255
            frame_cara_emotion = cv2.resize(gray_face, (105, 105)) / 255
            color_face_redim = cv2.resize(color_face, (96, 96))
            original_shape = gray_face.shape

            points, filter_dimensions = self.get_filter_dimensions(frame_cara)

            self.draw_points(points, x,y, w, h)

            if not filter_dimensions:
                continue

            ancho_gafas = filter_dimensions[0]
            alto_gafas = filter_dimensions[1]

            selfie_filter = self.get_filter(filter_image, points, color_face_redim, ancho_gafas, alto_gafas, original_shape)

            if selfie_filter is not None:
                frame[y:y + h, x:x + w] = selfie_filter

            if self.must_predict_emotion(hand_landmarks):
                self.last_emotion_predicted = self.predict_emotion_and_draw_emotion(frame_cara_emotion)
                print(self.last_emotion_predicted)


In [ ]:
if __name__ == '__main__':
    cap = cv2.VideoCapture(0)
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    handsCascade = cv2.CascadeClassifier('haarCascadeHands/cascadeHands.xml')

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    cont = 0
    last_filter_used = 'filtros/sunglasses.png'
    selfie_filters = ['filtros/sunglasses.png', 'filtros/sunglasses_2.png','filtros/sunglasses_3.jpg','filtros/sunglasses_4.png','filtros/sunglasses_5.jpg','filtros/moustache.png','filtros/sunglasses_6.png']

    setFiltersActions = SetFiltersActions()

    while True:
        # Capture frame-by-frame
        cap.set(cv2.CAP_PROP_FPS, 16)
        ret, frame = cap.read()

        # if frame is read correctly ret is True
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        # Our operations on the frame come here

        frame = cv2.flip(frame, 90)
        second_frame = frame.copy()

        setFiltersActions.second_frame = second_frame

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        setFiltersActions.draw_rectangles(frame)

        hand_landmarks = setFiltersActions.hand_detection(frame)

        if setFiltersActions.must_change_filter(hand_landmarks):
            setFiltersActions.random_filter = random.choice(selfie_filters)

        setFiltersActions.detect_faces(frame, hand_landmarks)

        cv2.putText(frame, setFiltersActions.last_emotion_predicted, (frame.shape[1] // 2, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                            (0, 0, 0), 3)
        # Display the resulting frame
        cv2.imshow('frame', frame)
        cv2.imshow('frame2', second_frame)
        if cv2.waitKey(1) == ord('q'):
            break

        cont += 1

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()
